In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
from utils.seed import fix_seed
import random
import os
from answer_judge.vqaEval import VQAEval

In [2]:
fix_seed(0)
data_reader=ReadData('/root/autodl-tmp/hallscope/InfoVQA/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,answer,question,most_likely,responses
0,[Bilingualism.],[What makes you smarter?\nWrite a very short a...,{'embedding': [[[ 0.00408936 -0.00958252 0.00...,None
1,"[45,811.]",[What is the number of scams reported in the U...,{'embedding': [[[ 1.8386841e-03 -1.4724731e-03...,None
2,[23.],[What % of australians have Asthma in Queensla...,{'embedding': [[[-0.01867676 0.00515747 0.00...,None
3,[50%.],[What percent of teenagers use smartphones for...,{'embedding': [[[ 0.00020885 -0.00279236 0.00...,None
4,[Kareem Abdul-Jabbar.],[Whose name is listed third among the names of...,{'embedding': [[[-0.00205994 -0.01599121 -0.02...,None


In [26]:
i=1
df.iloc[i]['most_likely']['response']

['45,811.']

In [27]:
df.iloc[i]['question']

['What is the number of scams reported in the U.S. in 2017?\nQuick response, please.\nAnswer the question using a single word or phrase.\n']

In [28]:
df.iloc[i]['answer']

['45,811.']

In [29]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [30]:
embeddings=np.concatenate(embeddings,axis=0)

In [31]:
embeddings.shape

(10074, 29, 3584)

In [32]:
gt_label=df['answer'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [33]:
gt_label.shape

(10074,)

In [34]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [35]:
from answer_judge import load_judge

In [36]:
is_correct=[]
judge=load_judge('InfoVQA')
for i,x in enumerate(resps):
    if judge.check(x,gt_label[i]):
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [37]:
is_correct.shape

(10074,)

In [38]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 10074 responses. 8636 answer correctly and 1438 answer wrongly


In [39]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:7059
validset length:1000
testset length:2015
trainset postive: 6069 negative: 990
valset postive: 838 negative: 162
testset postive: 1729 negative: 286


In [40]:
all_embeddings = embeddings
all_embeddings.shape

(10074, 29, 3584)

In [41]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [42]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel,KernelPCA
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from ml_utils.metrics import auc_pr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [43]:
M=3584
def evaluator(n_components,i_layer,X,y):
    kpca=PCALinear(X[:,i_layer,:],n_components)
    scores=kpca.get_score(X[:,i_layer,:])
    return roc_auc_score(y,scores)

In [44]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,4),
    'layer':range(10,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:1,layer:10 Score: 0.47680397183181594
found best n_components:1,layer:18 Score: 0.5013922036595068
found best n_components:1,layer:20 Score: 0.5533899054185449
found best n_components:1,layer:21 Score: 0.5717979315831344
found best n_components:1,layer:22 Score: 0.6006585344294174
found best n_components:1,layer:23 Score: 0.6214826600665901
found best n_components:1,layer:25 Score: 0.62793541353605


  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:2,layer:26 Score: 0.6283405521671234


  0%|          | 0/19 [00:00<?, ?it/s]

In [45]:
best_paras

{'n_components': 2, 'layer': 26, 'best_score': np.float64(0.6283405521671234)}

In [46]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [47]:
discriminator=KernelPCA(embedding_val[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [48]:
discriminator=KernelPCA(embedding_train[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.7333899985833687 AUROC:0.5923853795826114


In [49]:
discriminator=KernelPCA(embedding_test[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.5732009925558312 AUROC:0.5987514509781715


Train Linear Prob to Get Better Results

In [50]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [51]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [52]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)} auc-pr:{auc_pr(y_test,scores_test)}')

test set acc: 0.7205955334987593 f1:0.8313866427073974 auroc:0.5071911893774242  b-acc:0.5132761974867238 auc-pr:0.9071953449299344
